In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
import json 
from scipy import stats
from scipy.stats import linregress
from config import weather_api_key
#from config import g_key
from citipy import citipy
csvfile = "Buoy Stations - Sheet1.csv" 

In [2]:
buoy_df_raw = pd.read_csv(csvfile)
buoy_df = buoy_df_raw.dropna()
buoy_df.head()

,Buoy,Lat,Long
0,42002,26.055,-93.646
1,42003,25.925,-85.615
2,khhv,26.939,-94.689
3,kemk,27.819,-94.323
4,42019,27.910,-95.345


In [3]:
lat_val = []
lng_val = []
for index,row in buoy_df.iterrows():
    lat_val.append(row["Lat"])
    lng_val.append(row["Long"])

In [4]:
#set function to get today and previous 4 days' weather data
def unix_time():
    today = datetime.date.today()
    days = [1,2,3,4]
    times = [int(time.mktime(today.timetuple()))]
    for day in days:
        x=today - datetime.timedelta(days=day)
        unixtime = int(time.mktime(x.timetuple()))
        times.append(unixtime)
    return (times)

In [5]:
#lists for buoy data dataframe
hour_temp =[]
hour_pressure =[]
hour_wind_speed =[]
hour_wind_deg =[]
hour_time=[]
hour_lat=[]
hour_lng = []

In [6]:
for lat in range(len(lat_val)):
    for x in unix_time():
            historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat_val[lat]}&lon={lng_val[lat]}&dt={x}&appid={weather_api_key}&units=imperial"
            hist_json = requests.get(historical_url).json()
            for i in range((len(hist_json["hourly"]))):
                hour_temp.append(hist_json["hourly"][i]["temp"])
                hour_pressure.append(hist_json["hourly"][i]["pressure"])
                hour_wind_speed.append(hist_json["hourly"][i]["wind_speed"])
                hour_wind_deg.append(hist_json["hourly"][i]["wind_deg"])
                hour_time.append(hist_json["hourly"][i]["dt"])
                hour_lat.append(hist_json["lat"])
                hour_lng.append(hist_json["lon"])
      
        

In [7]:
buoy_dict = {"Lat":hour_lat,
            "Long":hour_lng,
            "Temp":hour_temp,
            "Pressure":hour_pressure,
            "Wind_Speed":hour_wind_speed,
            "Wind_Degrees":hour_wind_deg,
            "Unix Time":hour_time}
buoy_output_data = pd.DataFrame(buoy_dict)
buoy_output_data

,Lat,Long,Temp,Pressure,Wind_Speed,Wind_Degrees,Unix Time
0,26.055,-93.646,84.87,1013,6.11,90,1630800000
1,26.055,-93.646,84.76,1014,6.60,89,1630803600
2,26.055,-93.646,84.60,1014,6.49,95,1630807200
3,26.055,-93.646,84.60,1014,6.87,104,1630810800
4,26.055,-93.646,84.38,1014,7.07,115,1630814400
...,...,...,...,...,...,...,...
4789,21.030,-64.790,83.59,1014,4.88,134,1630522800
4790,21.030,-64.790,83.71,1014,4.41,129,1630526400
4791,21.030,-64.790,83.88,1014,5.75,121,1630530000
4792,21.030,-64.790,83.75,1014,7.25,123,1630533600


In [8]:
buoy_output_data.to_csv("./CSV_Output/buoy_data.csv", index=False)

In [9]:
#set "cit center" for Houston
Houston_lat = 29.756097853207656
Houston_lng = -95.3669907972107


#calculate greater Houston area by expanding lat and lng
lat = np.random.uniform(Houston_lat-1, Houston_lat+2, size=6500)
lng = np.random.uniform(Houston_lng-5, Houston_lng+2, size=6500)
city = []

#find nearest city using lat and lng
for x in range(len(lat)):
    city.append(citipy.nearest_city(lat[x], lng[x]).city_name)


In [37]:
#turn cities into df
df_city = pd.DataFrame({
    "City":city,
    "lat":lat,
    "lng":lng,
})

In [38]:
df_city

,City,lat,lng
0,port arthur,29.822248,-93.720622
1,lakeway,30.777566,-98.403497
2,santa fe,29.308168,-95.164774
3,eagle pass,28.799092,-100.349386
4,kerrville,30.256736,-99.393603
...,...,...,...
6495,richmond,29.520062,-95.721587
6496,copperas cove,31.359427,-98.411737
6497,hondo,28.999779,-99.370140
6498,pleasanton,28.776319,-98.439668


In [12]:
#check length of cities
# len(list(df_city["City"].unique()))

In [39]:
cities = list(df_city["City"].unique())
target_cities=[]

for i in cities:
    c=str(i).replace(" ","+")
    target_cities.append(c)

In [43]:
#set base url and define function to find OpenWeather cities' based on city name from nearest place API and feed back into
#OpenWeather one call API to ensure proper lat and lng

def find_location(target_cities):
    unix_time()
    base_current_url = "http://api.openweathermap.org/data/2.5/weather?"
    hour_temp_city =[]
    hour_pressure_city =[]
    hour_wind_speed_city =[]
    hour_wind_deg_city=[]
    hour_time_city=[]
    city_name=[]
    for city in target_cities:
        try: 
            query_url=f"{base_current_url}appid={weather_api_key}&q={city},Texas,USA&units=imperial"
            req = requests.get(query_url)
            response1=req.json()
            latitude=response1["coord"]["lat"]
            longitude=response1["coord"]["lon"]

        except:
            print("City not found. Skipping...")
            pass
        if response1["sys"]["country"]=="US":
            for x in unix_time():
                historical_url = f"http://api.openweathermap.org/data/2.5/onecall/timemachine?lat={latitude}&lon={longitude}&dt={x}&appid={weather_api_key}&units=imperial"
                hist_json = requests.get(historical_url).json()
                for i in range(len(hist_json["hourly"])):
                    hour_temp_city.append(hist_json["hourly"][i]["temp"])
                    hour_pressure_city.append(hist_json["hourly"][i]["pressure"])
                    hour_wind_speed_city.append(hist_json["hourly"][i]["wind_speed"])
                    hour_wind_deg_city.append(hist_json["hourly"][i]["wind_deg"])
                    hour_time_city.append(hist_json["hourly"][i]["dt"])
                    city_name.append(response1["name"])
                
    d=dict()
    d["City Name"] = city_name
    d["Temperature"] = hour_temp_city
    d["Pressure"] = hour_pressure_city
    d["Wind Degrees"]=hour_wind_deg_city
    d["Wind_Speed"] = hour_wind_speed_city
    return(d)

In [44]:
df2 =pd.DataFrame(find_location(target_cities))

In [45]:
df2

,City Name,Temperature,Pressure,Wind Degrees,Wind_Speed
0,Port Arthur,86.67,1012,210,6.91
1,Port Arthur,84.22,1012,210,5.75
2,Port Arthur,83.43,1013,200,5.75
3,Port Arthur,81.61,1013,230,3.44
4,Port Arthur,80.49,1013,220,4.61
...,...,...,...,...,...
11459,Pasadena,70.93,1012,0,0.00
11460,Pasadena,72.32,1012,0,0.00
11461,Pasadena,73.06,1012,0,3.44
11462,Pasadena,74.37,1012,0,0.00


In [46]:
df2.to_csv("./CSV_Output/city_data.csv", index=False)